In [1]:
import pandas as pd
import requests
import time
from pathlib import Path

In [ ]:
# === Config ===
MODEL = "model-1"
INPUT_FILE = "../../data/processed/exp002.csv"
EVAL_FILE = "eval.csv"
RESULT_FILE = "../../results/exp002/results.csv"
TEMPERATURE = 0.0
N_PER_CLASS = 50

In [ ]:
# === Step 1: Prepare eval dataset (balanced) ===
df = pd.read_csv(INPUT_FILE).dropna(subset=["Manchete"])
df_up = df[df["Direção"] == "Aumento"].sample(n=N_PER_CLASS, random_state=42)
df_down = df[df["Direção"] == "Diminuição"].sample(n=N_PER_CLASS, random_state=42)
eval_df = pd.concat([df_up, df_down]).sample(frac=1.0, random_state=42).reset_index(drop=True)
eval_df.to_csv(EVAL_FILE, index=False)

In [ ]:
# === Step 2: Prompt generator ===
def build_prompt(headline, timestamp):
    return f"""Manchete: {headline}
DataHora: {timestamp}

Responda apenas com uma palavra: Aumento ou Diminuição."""

In [ ]:
# === Step 3: Query Ollama ===
def query_ollama(model, prompt):
    res = requests.post("http://localhost:11434/api/chat", json={
        "model": model,
        "temperature": TEMPERATURE,
        "stream": False,
        "messages": [{"role": "user", "content": prompt}]
    })
    res.raise_for_status()
    return res.json()["message"]["content"].strip()

In [ ]:
# === Step 4: Run prediction loop ===
predictions = []
for i, row in eval_df.iterrows():
    prompt = build_prompt(row["Manchete"], row["DataHora"])
    try:
        out = query_ollama(MODEL, prompt)
    except Exception as e:
        out = "Erro"
    predictions.append(out)
    time.sleep(0.25)  # reduce overload risk

In [ ]:
eval_df["Previsão"] = predictions
eval_df.to_csv(RESULT_FILE, index=False)
print(f"Saved predictions to {RESULT_FILE}")

In [5]:
from sklearn.metrics import classification_report

# Reload the results file
df = pd.read_csv("../../results/exp002/results.csv")

# Normalize predictions (just in case)
df["Previsão"] = df["Previsão"].str.strip().str.capitalize()
df["Direção"] = df["Direção"].str.strip().str.capitalize()

# Report
report = classification_report(
    df["Direção"], df["Previsão"],
    labels=["Aumento", "Diminuição"],
    target_names=["Aumento", "Diminuição"],
    digits=3
)

print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

     Aumento      0.446     0.580     0.504        50
  Diminuição      0.424     0.280     0.337        50

   micro avg      0.439     0.430     0.434       100
   macro avg      0.435     0.430     0.421       100
weighted avg      0.435     0.430     0.421       100



In [ ]:
from sklearn.metrics import confusion_matrix
'''
[[TP FN]
 [FP TN]]
'''
cm = confusion_matrix(df["Direção"], df["Previsão"], labels=["Aumento", "Diminuição"])
print(cm)

[[29 19]
 [36 14]]
